# Setting up the initial simulation grid


### Simulation parameters $H$

From each simulation we run, we will output a number of snapshots, of order 10. 

Moreover, for each snapshot we extract different sets of simulated Lyman-$\alpha$ skewers (normalized quasar spectra), after applying different rescalings of the temperatures in the snapshot. 
These skewers are written to disk.

Finally, from each set of skewers we measure different 1D power spectra, after rescaling the mean optical depth in the spectra. 
This optical depth rescaling is trivial, and it is done on the fly. 
Each of these measured power spectra is fed to the emulator. 


Let's discuss the parameterization of each of the simulation packages:

- As we discuss above, the emulator labels the measured power with the set of parameters $ M = \{ M_P, f, \lambda_F, \bar F, \sigma_T, \gamma \} $.

- Each set of simulated skewers, written to disk, is described by a subset of these $ \{ M_P, f, \lambda_F, \sigma_T, \gamma \} $, since we will have different values of $\bar F$ from the skewers.

- Each snapshot is described by an even smaller subset of parameters, $ \{ M_P, f, \lambda_F \} $, since we will reprocess the snapshot for different temperature-density relations.

If we assumed that we can do as much rescaling of mean flux and temperature as we wanted, then this last set of parameters $ \{ M_P, f, \lambda_F \} $ would be the only relevant ones. If we can not rescale as much as we would like to (because the rescaling breaks down at some point?), then we would need to label the snapshots with a "central temperature" and "central mean flux", around which we would perturb. 

Now, since we are describing the linear power in comoving units, the shape of the linear power will be the same in all snapshots. The only parameters that will vary are the amplitude of the linear power, the logarithmic growth rate $f$ and the filtering length $\lambda_F$. So each simulation will be described by two shape parameters, and $N_z \times \{ A_p, f, \lambda_F \}$, where $N_z$ is the number of snapshots. 

$$ H = \{ n_p, \alpha_p, N_z \times \{ A_p, f, \lambda_F \} \} $$


The question is: how do we decide what simulation to run, i.e., what configuration files to use, if we are handed a set of parameters $H$? We will not aim at having a perfect match between the two, we just want to get on the right ball park.

From the point of view of cosmological parameters:
 - We start by defining a fiducial cosmology, somewhere around Planck, but WITHOUT NEUTRINOS. This is important, since our simulations will not have neutrinos. All quantities with $^0$ will refer to that fiducial cosmology. 
 - In flat $\Lambda$CDM models, we can translate the requirement of a given $f_\star$ into a requirement of a given $\Omega_\star = \Omega_{cb}(z_\star)$, using $f_\star = \Omega_\star^{0.55}$.
 - We can then translate the requirement on $\Omega_\star$ to a requirement on $\Omega_{cb} = \Omega_{cb}(z=0)$, using $\Omega_\star = \Omega_{cb} (1+z_\star)^3 / ( \Omega_{cb} (1+z_\star)^3 + 1 - \Omega_{cb} ) $. 
 - We will keep fixed the values of $\Omega_c h^2$ and $\Omega_b h^2$, so the requirement on $\Omega_{cb}$ can be related to a requirement on $h$, using $h^2 = (\Omega_b h^2 + \Omega_c h^2)/\Omega_{cb} $.
 - We will also measure the shape of the linear power at $z_\star$ around $k_p$, and modify the slope and running of the primordial power to match the required shape.
 - So the cosmology in the simulation will be specified by $\{ \Omega_\star, A_p, n_p, \alpha_p \}$, where $A_p$ will be computed at $z_\star$.

From the point of view of the pressure smoothing:
 - We will look at the value of the filtering length at a pre-HeII and at a post-HeII redshift, say z=2 and z=4. 
 - We will use the Oñorbe models to translate the required values for $\lambda_F$ pre- and post-HeII reionization to a required thermal history. It will probably be enough to specify a scaling of the heating rates before ($\mu_H$) and after ($\mu_{He}$) He_II reionizaion. 

### Initial simulation grid

We have 6 simulation parameters: 

$$ H = \{ \mu_H, \mu_{He}, \Omega_\star, \Delta_p^2, n_p, \alpha_p \} $$

We will choose the following parameter volume for the simulations:
 - $ 0.5 < \mu_H < 2.0 $
 - $ 0.5 < \mu_{He} < 2.0 $
 - $ 0.950 < \Omega_\star < 0.975 $
 - $ 0.25 < \Delta_p^2 < 0.45 $
 - $ -2.4 < n_p < -2.3 $
 - $ -0.12 < \alpha_p < -0.08 $

In [1]:
%matplotlib inline
#%load_ext autoreload
#%autoreload 2
import numpy as np
import os
import copy
## Set default plot size, as normally its a bit too small
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 120
mpl.rcParams['figure.dpi'] = 120
import fit_pk
import read_genic
import write_genic
import camb_cosmo
import latin_hypercube

In [2]:
def get_sim_params(z_star=3.0,kp_Mpc=0.7):
    """Return list of simulation parameters, with name and allowed range"""
    params={}
    params['mu_H']={'ip':0, 'min_val':0.5, 'max_val':2.0}
    params['mu_He']={'ip':1, 'min_val':0.5, 'max_val':2.0}
    params['Om_star']={'ip':2, 'min_val':0.950, 'max_val':0.975, 'z_star':z_star}
    params['Delta2_star']={'ip':3, 'min_val':0.25, 'max_val':0.45, 'z_star':z_star, 'kp_Mpc':kp_Mpc}
    params['n_star']={'ip':4, 'min_val':-2.4, 'max_val':-2.3, 'z_star':z_star, 'kp_Mpc':kp_Mpc}
    params['alpha_star']={'ip':5, 'min_val':-0.12, 'max_val':-0.08, 'z_star':z_star, 'kp_Mpc':kp_Mpc}
    return params

In [3]:
def Om_from_Om_star(Om_star,z_star=3):
    z3=(1+z_star)**3
    Om=Om_star/(z3+Om_star-Om_star*z3)
    return Om

In [4]:
def cosmo_from_sim_params(param_values,cosmo_fid,z_star,kp_Mpc):
    """Given list of simulation parameters, and fiducial cosmology, return target cosmology"""
    # get linear power parameters, in comoving units
    linP_params_fid=fit_pk.parameterize_cosmology_Mpc(cosmo_fid,z_star=z_star,kp_Mpc=kp_Mpc)
    print('fiducial cosmology parameters',linP_params_fid)
    # translate Omega_star to Omega_0
    Om_star=params_values[2]
    Om=Om_from_Om_star(params[2],z_star=z_star)
    print('Omega_m',Om)

- Setup Latin hypercube

In [5]:
# get list of simulation parameters
z_star=3.0
kp_Mpc=0.7
params = get_sim_params(z_star=z_star,kp_Mpc=kp_Mpc)
Npar=len(params)
param_limits=np.empty([Npar,2])
for key,param in params.items():
    ip=param['ip']
    print("{} = {}".format(key, ip))
    param_limits[ip][0]=param['min_val']
    param_limits[ip][1]=param['max_val']

mu_H = 0
mu_He = 1
Om_star = 2
Delta2_star = 3
n_star = 4
alpha_star = 5


In [6]:
# generate Latin hypercube 
nsamples=5
seed=123
cube=latin_hypercube.get_hypercube_samples(param_limits, nsamples, prior_points = None, seed=seed)
print('first point in cube',cube[0])

first point in cube [ 0.95    0.95    0.9575  0.39   -2.35   -0.084 ]


 - Setup fiducial cosmology

In [7]:
cosmo_fid = camb_cosmo.get_cosmology()
camb_cosmo.print_info(cosmo_fid)
# get linear power parameters, in comoving units
linP_params_fid=fit_pk.parameterize_cosmology_Mpc(cosmo_fid,z_star=z_star,kp_Mpc=kp_Mpc)
print('fiducial cosmology parameters',linP_params_fid)

H0 = 67.0 ; Omega_b h^2 = 0.022 ; Omega_c h^2 = 0.12 ; Omega_k = 0.0 ; ommnuh2 = 0.0 ; T_CMB = 2.7255 ; A_s = 2.1e-09 ; n_s = 0.96 ; alpha_s = 0.0
fiducial cosmology parameters {'f_star': 0.98136698840516934, 'g_star': 0.9677508579459803, 'linP_Mpc': poly1d([-0.10748482, -2.30758676,  3.02860938])}


In [8]:
point=cube[0]
print('first point in cube',point)
# extract relevant information from point
ip_Om_star=params['Om_star']['ip']
Om_star=point[ip_Om_star]
print(ip_Om_star,Om_star)
# translate Omega_star to Omega_0
Om=Om_from_Om_star(Om_star,z_star)
print('Omega_m',Om)

first point in cube [ 0.95    0.95    0.9575  0.39   -2.35   -0.084 ]
2 0.9575
Omega_m 0.260367097213


In [9]:
# get parameters from fiducial cosmology
Obh2=cosmo_fid.ombh2
Och2=cosmo_fid.omch2
Omh2=Obh2+Och2
h=np.sqrt(Omh2/Om)
print('h=',h)
# get temporary cosmology to tune primordial power spectrum
cosmo_temp=camb_cosmo.get_cosmology(H0=100.0*h)
# get linear power parameters, in comoving units
linP_params_temp=fit_pk.parameterize_cosmology_Mpc(cosmo_temp,z_star=z_star,kp_Mpc=kp_Mpc)
print('temporary cosmology parameters',linP_params_temp)

h= 0.738501057555
temporary cosmology parameters {'f_star': 0.97591366253738121, 'g_star': 0.9579239654271418, 'linP_Mpc': poly1d([-0.10744778, -2.30756118,  3.02486379])}


In [10]:
# difference in linear power at kp between target and fiducial cosmology, once they have the same transfer function
ip_Delta2_star=params['Delta2_star']['ip']
Delta2_star=point[ip_Delta2_star]
A_star=Delta2_star*(2*np.pi**2)/kp_Mpc**3
lnA_star=np.log(A_star)
delta_lnA_star=lnA_star-linP_params_temp['linP_Mpc'][0]
print('delta_lnA_star',delta_lnA_star)
# slope
ip_n_star=params['n_star']['ip']
n_star=point[ip_n_star]
delta_n_star=n_star-linP_params_temp['linP_Mpc'][1]
print('delta_n_star',delta_n_star)
# running
ip_alpha_star=params['alpha_star']['ip']
alpha_star=point[ip_alpha_star]
delta_alpha_star=alpha_star-linP_params_temp['linP_Mpc'][2]
print('delta_alpha_star',delta_alpha_star)

delta_lnA_star 0.086159459143
delta_n_star -0.0424388199641
delta_alpha_star 0.0234477821373


In [11]:
# transform differences into differences at kp_CMB
kCMB=cosmo_fid.InitPower.pivot_scalar
# ratio of pivot points
ln_kCMB_p=np.log(kCMB/kp_Mpc)
print('ln_kCMB_p',ln_kCMB_p)
delta_nrun=delta_alpha_star
delta_ns=delta_n_star+delta_alpha_star*ln_kCMB_p
delta_lnAs=delta_lnA_star+delta_n_star*ln_kCMB_p+0.5*delta_alpha_star*ln_kCMB_p**2
print('delta_lnAs',delta_lnAs)
print('delta_ns',delta_ns)
print('delta_nrun',delta_nrun)

ln_kCMB_p -2.63905732962
delta_lnAs 0.279810426321
delta_ns -0.104318861277
delta_nrun 0.0234477821373


In [12]:
# compute primordial power for target cosmology
lnAs=np.log(cosmo_temp.InitPower.As)+delta_lnAs
As=np.exp(lnAs)
ns=cosmo_temp.InitPower.ns+delta_ns
nrun=cosmo_temp.InitPower.nrun+delta_nrun
print('As',np.exp(lnAs))
print('ns',ns)
print('nrun',nrun)

As 2.7780459116e-09
ns 0.855681138723
nrun 0.0234477821373


In [13]:
# setup simulation cosmology object
cosmo_sim=camb_cosmo.get_cosmology(H0=100.0*h,As=As,ns=ns,nrun=nrun)
camb_cosmo.print_info(cosmo_sim)
# get linear power parameters, in comoving units
linP_params_sim=fit_pk.parameterize_cosmology_Mpc(cosmo_sim,z_star=z_star,kp_Mpc=kp_Mpc)
print('simulation cosmology parameters',linP_params_sim)
print('sim Delta2 =',np.exp(linP_params_sim['linP_Mpc'][0])*kp_Mpc**3/(2*np.pi**2))
print('target point was',point)

H0 = 73.85010575553991 ; Omega_b h^2 = 0.022 ; Omega_c h^2 = 0.12 ; Omega_k = 0.0 ; ommnuh2 = 0.0 ; T_CMB = 2.7255 ; A_s = 2.7780459116026874e-09 ; n_s = 0.8556811387232228 ; alpha_s = 0.0234477821373157
simulation cosmology parameters {'f_star': 0.97591366253738121, 'g_star': 0.9579239654271418, 'linP_Mpc': poly1d([-0.09572389, -2.35      ,  3.11102324])}
sim Delta2 = 0.39
target point was [ 0.95    0.95    0.9575  0.39   -2.35   -0.084 ]


 - Write simulation config files

In [14]:
root_file_name='test_sim'
write_genic.write_genic_file(root_file_name,cosmo_sim)
write_genic.write_gadget_file(root_file_name,cosmo_sim)
write_genic.write_json_file(root_file_name,cosmo_sim,point)

Modify GenIC to allow for running 0.0234477821373157
filename  test_simjson
H0
